## Setup

Load libraries

In [4]:
import os
import shap
import pickle
import numpy as np
import pandas as pd

from utils.model_fitting_utils import ModelFitting
from utils.features import UAVSAR_FEATURES, UAVSAR_AND_VEGETATION_HEIGHT_FEATURES

## Load Data

In [7]:
with open('../data/full_splits.pkl', 'rb') as f:
    full_splits = pickle.load(f)


with open('../data/vegetation_splits.pkl', 'rb') as f:
    vegetation_splits = pickle.load(f)


with open('../data/no_vegetation_splits.pkl', 'rb') as f:
    no_vegetation_splits = pickle.load(f)

## Overall Best Model (XGBoost for the combined dataset)

In [8]:
xgboost_utils = {
        "sampling_method": "gradient_based",
        'objective': 'reg:squarederror',
        "min_child_weight": 30,
        'learning_rate': 0.05,
        'tree_method': 'hist',
        'booster': 'gbtree',
        'device': 'cuda',
        'max_depth': 0,
        "subsample": 1,
        "max_bin":5096,
        "trees": 50,
        "seed": 42
    }

In [9]:
dtrain=xgb.DMatrix(full_splits['X_temp'][UAVSAR_AND_VEGETATION_HEIGHT_FEATURES], label=full_splits['y_temp'])
n_trees = xgboost_utils["trees"]
boosting_params = xgboost_utils.copy()
boosting_params.pop("trees")

50

In [10]:
model = xgb.train(
    params=boosting_params,
    dtrain=dtrain,
    num_boost_round=n_trees
)

In [11]:
# model = xgb.Booster()
# model.load_model("model_weights_all2.ubj")  # Load model from Universal Binary JSON file

# ## Calculate SHAP values for the density model
# model.set_param({"device": "cuda"}) ## comment this out if you do not have a GPU
explainer = shap.TreeExplainer(model)

shap_values = explainer.shap_values(full_splits['X_test'][UAVSAR_AND_VEGETATION_HEIGHT_FEATURES])

In [12]:
shap_importance_combined=(
    pd.DataFrame(
        data={
            "Feature": model.feature_names,
            "Importance": np.abs(shap_values).mean(axis=0)
        }
    )
    .sort_values(by="Importance", ascending=False)
)

shap_importance_combined

,Feature,Importance
6,unwrapped_phase,0.086921
4,elevation,0.073468
1,amplitude,0.052275
5,incidence_angle,0.049994
2,vegetation_ht,0.032201
3,wrapped_phase,0.018315
0,coherence,0.011917


## Open Areas

In [13]:
with open('no_vegetation_splits.pkl', 'rb') as f:
    no_vegetation_splits = pickle.load(f)

In [14]:
dtrain=xgb.DMatrix(no_vegetation_splits['X_temp'][UAVSAR_FEATURES], label=no_vegetation_splits['y_temp'])

model = xgb.train(
    params=boosting_params,
    dtrain=dtrain,
    num_boost_round=50
)

In [15]:
# model.set_param({"device": "cuda"}) ## comment this out if you do not have a GPU
explainer = shap.TreeExplainer(model)

shap_values_open = explainer.shap_values(no_vegetation_splits['X_test'][UAVSAR_FEATURES])

In [16]:
shap_importance_open=(
    pd.DataFrame(
        data={
            "Feature": model.feature_names,
            "Importance": np.abs(shap_values_open).mean(axis=0)
        }
    )
    .sort_values(by="Importance", ascending=False)
)

shap_importance_open

,Feature,Importance
5,unwrapped_phase,0.120974
3,elevation,0.064700
4,incidence_angle,0.037485
1,amplitude,0.028049
2,wrapped_phase,0.025552
0,coherence,0.013002


## Vegetated Areas

In [17]:
with open('vegetation_splits.pkl', 'rb') as f:
    vegetation_splits = pickle.load(f)

In [18]:
dtrain=xgb.DMatrix(vegetation_splits['X_temp'][UAVSAR_AND_VEGETATION_HEIGHT_FEATURES], label=vegetation_splits['y_temp'])

model = xgb.train(
    params=boosting_params,
    dtrain=dtrain,
    num_boost_round=50
)

In [20]:
explainer = shap.TreeExplainer(model)

shap_values_vegetated = explainer.shap_values(vegetation_splits['X_test'][UAVSAR_AND_VEGETATION_HEIGHT_FEATURES])

In [21]:
shap_importance_vegetated=(
    pd.DataFrame(
        data={
            "Feature": model.feature_names,
            "Importance": np.abs(shap_values_vegetated).mean(axis=0)
        }
    )
    .sort_values(by="Importance", ascending=False)
)

shap_importance_vegetated

,Feature,Importance
4,elevation,0.099073
5,incidence_angle,0.084523
2,vegetation_ht,0.063105
6,unwrapped_phase,0.031785
1,amplitude,0.009196
3,wrapped_phase,0.006391
0,coherence,0.005100


In [22]:
shap_importance_vegetated.to_csv("vegetated_shap.csv", index=False)

In [23]:
shap_importance_open.to_csv("open_shap.csv", index=False)

In [24]:
shap_importance_combined.to_csv("combined_shap.csv", index=False)